In [1]:
%matplotlib inline
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
from  ggplot import *

/home/ramon/anaconda2/lib/python2.7/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/ramon/anaconda2/lib/python2.7/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/ramon/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Read all files names

In [2]:
col_interest =['id', 'host_id', 'host_since',
       'host_listings_count', 'host_total_listings_count',
       'property_type', 'room_type', 'bathrooms',
       'bedrooms', 'beds', 'price', 'calendar_updated', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'reviews_per_month']

In [41]:
def combine_cities():
    local_files =  [join('data/benchmark_capitals/',f) for f in listdir('data/benchmark_capitals/') if isfile(join('data/benchmark_capitals/', f))]
    local_files = [f for f in local_files if "all_" not in f]
    df_all_cities = pd.DataFrame()
    print local_files

    
    for cities in local_files:
        city_name =  cities.split('_')[2].split('.')[0]
        df = pd.read_csv(cities)
        df = df[col_interest]
        df.price = df.price.astype(str)
        df.price= df.price.str.replace('[$,]','')
        df['city']=city_name.title()
        df_all_cities =pd.concat([df_all_cities,df],axis=0)
    return df_all_cities

In [42]:
df = combine_cities()
df.to_csv('data/benchmark_capitals/all_listings.csv',index=False)

['data/benchmark_capitals/listings_barcelona.csv', 'data/benchmark_capitals/listings_antwerp.csv', 'data/benchmark_capitals/listings_athens.csv', 'data/benchmark_capitals/listings_madrid.csv', 'data/benchmark_capitals/listings_london.csv', 'data/benchmark_capitals/listings_geneva.csv', 'data/benchmark_capitals/listings_venice.csv', 'data/benchmark_capitals/listings_roma.csv', 'data/benchmark_capitals/listings_dublin.csv', 'data/benchmark_capitals/listings_amsterdam.csv', 'data/benchmark_capitals/listings_vienna.csv', 'data/benchmark_capitals/listings_paris.csv', 'data/benchmark_capitals/listings_berlin.csv', 'data/benchmark_capitals/listings_brussels.csv']


In [40]:
np.sum(pd.to_numeric(df.price).isnull())


0

In [27]:
df[df['id']==15395696]

,id,host_id,host_since,host_listings_count,host_total_listings_count,property_type,room_type,bathrooms,bedrooms,beds,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,city
18147,15395696,2164,2008-08-16,2.0,2.0,Loft,Entire home/apt,1.0,0.0,1.0,...,2017-04-15,70.0,10.0,8.0,10.0,9.0,8.0,10.0,1.54,berlin


In [6]:
df.shape

(264795, 28)

In [7]:
def is_professional(x):
    return (np.mean((x!=0)&(x<25))*100.0)

In [9]:
df_out = df.groupby('room_type').agg({'availability_30':{"mean_availability_30":np.mean, 'ratio_professional': is_professional}, "reviews_per_month":{'average_reviews_per_month':'mean'}, 'price':{'mean_price':'mean'}})[['availability_30','reviews_per_month','price']]

/home/ramon/anaconda2/lib/python2.7/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [10]:
df_out.columns = df_out.columns.droplevel(0)
df_out

,ratio_professional,mean_availability_30,average_reviews_per_month,mean_price
room_type,,,,
Entire home/apt,72.093810,5.862104,1.450000,138.252678
Private room,57.074703,9.436118,1.669483,45.480451
Shared room,52.631579,15.447368,1.287581,45.710526


In [11]:
df_berlin = pd.read_csv('data/benchmark_capitals/listings_berlin.csv')

In [ ]:
pd.to_datetime(df_berlin.host_since).hist()

In [ ]:
(pd.to_datetime(df_berlin.host_since)).idxmin()

In [12]:
df_berlin.iloc[18147]

id                                                                           15395696
listing_url                                     https://www.airbnb.com/rooms/15395696
scrape_id                                                              20170507222235
last_scraped                                                               2017-05-08
name                                 spacious and bright artist's live-in studio loft
summary                             You’ll love my place because of the high ceili...
space                               One large space of 70 m2, bright, friendly and...
description                         You’ll love my place because of the high ceili...
experiences_offered                                                              none
neighborhood_overview                                                             NaN
notes                                                                             NaN
transit                             The loft is close 

In [ ]:
df_berlin.host_since.max()

In [ ]:
np.mean(df_berlin.availability_30.isnull())

In [ ]:
df_berlin['occupancy_rate'] = (30-df_berlin[df_berlin.availability_30>0].availability_30)*100.0//30.0
df_berlin.occupancy_rate.hist()

In [ ]:
ggplot(df_berlin,aes(x='occupancy_rate',y='reviews_per_month'))+ geom_boxplot()

In [ ]:
df_berlin.last_review = pd.to_datetime(df_berlin.last_review)

In [ ]:
t_zero =df_berlin.last_review.max()

In [ ]:
df_berlin['days_since_last_review']= (t_zero-df_berlin.last_review).dt.days

In [ ]:
df_berlin['is_active_host']= df_berlin.days_since_last_review<10

In [ ]:
df_berlin.groupby('room_type')[['is_active_host']].mean()

In [ ]:
pd.DataFrame(df_berlin.isnull().sum()/len(df_berlin)*100,columns=['ratio_missing']).sort_values(by=)

In [ ]:
df_berlin_cleansed = pd.read_csv('data/listing_cleansed.csv')
df_berlin_cleansed.shape